In [0]:
# Databricks notebook source
# ==========================================================
# UNIFICAR Y LIMPIAR PARQUETS HISTÓRICOS (1FL)
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F
import traceback

# Importar utilidades ADLS
from util.utils_liga1 import setup_adls, get_abfss_path, get_dbutils

# COMMAND ----------
# 📦 PARAMETROS DESDE ADF
dbutils.widgets.text("filesystem", "")
dbutils.widgets.text("capa_raw", "")
dbutils.widgets.text("rutaBase", "")
dbutils.widgets.text("nombre_archivo", "")
dbutils.widgets.text("historical_start_year", "")
dbutils.widgets.text("current_year", "")

filesystem = dbutils.widgets.get("filesystem")
capa_raw = dbutils.widgets.get("capa_raw").strip("/")
rutaBase = dbutils.widgets.get("rutaBase").strip("/")
nombre_archivo = dbutils.widgets.get("nombre_archivo")
historical_start_year = int(dbutils.widgets.get("historical_start_year"))
current_year = int(dbutils.widgets.get("current_year"))

spark = SparkSession.builder.getOrCreate()

# COMMAND ----------
# ⚙️ Configurar conexión a ADLS
setup_adls()
base_path = get_abfss_path(f"{capa_raw}/{rutaBase}/{nombre_archivo}")
dbutils = get_dbutils()

print("===============================================")
print("  🧩 UNIFICADOR HISTÓRICO DE PARQUETS (1FL)")
print("===============================================")
print(f"Entidad        : {nombre_archivo}")
print(f"Años procesados: {historical_start_year} - {current_year}")
print(f"Ruta base RAW  : {base_path}")
print("===============================================")

In [0]:
# 🔍 Buscar carpetas temporales con datos Parquet
rutas_temp = []
for year in range(historical_start_year, current_year + 1):
    ruta = f"{base_path}/temp/{year}/data"
    try:
        archivos = dbutils.fs.ls(ruta)
        if any(f.name.endswith(".parquet") for f in archivos):
            rutas_temp.append(ruta)
    except Exception:
        print(f"⚠️ Carpeta no encontrada o vacía: {ruta}")

if not rutas_temp:
    raise Exception("❌ No se encontraron archivos temporales para unificar.")

print(f"📂 Carpetas detectadas: {len(rutas_temp)}")
for r in rutas_temp:
    print(f" - {r}")

# COMMAND ----------
# 🧮 Consolidar DataFrames con esquema flexible
df_final = None
columnas_ref = None

for ruta in rutas_temp:
    try:
        df = spark.read.parquet(ruta)
        print(f"✅ Leído correctamente: {ruta}")

        # Inicializar columnas de referencia
        if columnas_ref is None:
            columnas_ref = df.columns
        else:
            faltantes = [c for c in columnas_ref if c not in df.columns]
            nuevos = [c for c in df.columns if c not in columnas_ref]

            for c in faltantes:
                df = df.withColumn(c, F.lit(None))
            for n in nuevos:
                if n not in columnas_ref:
                    columnas_ref.append(n)

            df = df.select(columnas_ref)

        # Append
        df_final = df if df_final is None else df_final.unionByName(df, allowMissingColumns=True)

    except AnalysisException as e:
        print(f"💥 Error leyendo {ruta}: {e.desc}")
    except Exception:
        print(f"💥 Error general en {ruta}: {traceback.format_exc()}")

if df_final is None:
    raise Exception("❌ No se pudo generar DataFrame consolidado.")

total_filas = df_final.count()
print(f"✅ Total filas unificadas: {total_filas}")

# COMMAND ----------
# 💾 Guardar parquet consolidado final
output_path = f"{base_path}/1FL/data"
final_file = f"{output_path}/{nombre_archivo}.parquet"

print("===============================================")
print(f"💾 Guardando consolidado final en: {final_file}")
print("===============================================")

(
    df_final
    .withColumn("fecha_unificacion", F.current_timestamp())
    .write
    .mode("overwrite")
    .parquet(final_file)
)

print(f"✅ Consolidado guardado con éxito ({total_filas} registros).")

# COMMAND ----------
# 🧹 Eliminar carpetas temporales tras consolidar
print("===============================================")
print("🧹 Iniciando limpieza de carpetas temporales...")
print("===============================================")

for ruta in rutas_temp:
    try:
        dbutils.fs.rm(ruta, True)
        print(f"🧼 Eliminada: {ruta}")
    except Exception as e:
        print(f"⚠️ No se pudo eliminar {ruta}: {e}")

print("✅ Limpieza completada correctamente.")
print("🎯 Proceso de unificación histórica finalizado con éxito.")